In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from classifierModel import EncoderModel, ClassifierModel
import torch
import numpy as np
import os
import pandas as pd
import spacy
import sys
import torch.nn as nn
import torch.nn.functional as F
import pickle

from data_utils import (IndexVectorizer,
                        SpacyTokenizer,
                        TextDataset,
                        LMDataLoader,
                        CLFDataLoader)

In [7]:
BATCH_SIZE = 50

# GPU setup
use_gpu = torch.cuda.is_available()
device_num = 0
device = torch.device(f"cuda:{device_num}" if use_gpu else "cpu")
print(device)

cuda:0


In [8]:
DATA_DIR = '../data/imdb/'
TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
VALID_PATH = os.path.join(DATA_DIR, 'valid.csv')

train = pd.read_csv(TRAIN_PATH)
valid = pd.read_csv(VALID_PATH)

vectorizer = pickle.load(open("lm_vectorizer.pkl", "rb"))
train_ds = TextDataset(data=train, vectorizer=vectorizer, 
                       text_col='text', label_col='label')
valid_ds = TextDataset(data=valid, vectorizer=vectorizer, 
                       text_col='text', label_col='label')

In [9]:
train_dl = CLFDataLoader(dataset=train_ds, batch_size=BATCH_SIZE)
valid_dl = CLFDataLoader(dataset=valid_ds, batch_size=BATCH_SIZE)

In [10]:
# Load the language model weights  
# but remove the decoder 
d = torch.load(DATA_DIR+"models/LM__2019-02-01.json")
del d["decoder.weight"]
del d["decoder.bias"]

In [11]:
# Generate the (document)Encoder model 
# from the language model weights
embedding_size = d['encoder.weight'].shape[1]
hidden_size = d['lstm1.weight_hh_l0'].shape[1]
m = EncoderModel(vectorizer, hidden_size, embedding_size, bidirectional=False)
m.load_state_dict(d)
m.requires_grad = False

In [15]:
# Put it together with the classification head
c = ClassifierModel(lm_hidden_size=hidden_size, hidden_size=200, output_size=3)
final = nn.Sequential(m,c)
if use_gpu:
    final = final.to(device)

In [16]:
def get_accuracy(pred_probs, true_class):
    '''Calculates average accuracy over batch'''
    pred_class = torch.argmax(pred_probs, dim=1)
    errors = pred_class == y
    return torch.mean(errors.type(torch.float)).item()

In [ ]:
optimizer = torch.optim.Adam(final.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

n_epochs = 100

for epoch in range(n_epochs):
    final.train()
    epoch_train_accs = []
    for x, y in train_dl:
        x = x.to(device)
        y = y.to(device)
        final.zero_grad()
        res = final(x)
        error = criterion(res, y)
        error.backward()
        optimizer.step()
        epoch_train_accs.append(get_accuracy(res, y))
        del error
        
    # Validation accuracy
    with torch.no_grad():
        final.eval()
        epoch_train_acc = round(np.mean(epoch_train_accs), 3)
        valid_accs = []
        for x, y in valid_dl:
            x = x.to(device)
            y = y.to(device)
            pred_prob = final(x)
            valid_accs.append(get_accuracy(pred_prob, y))
        valid_acc = round(np.mean(valid_accs), 3)
        print(f'Epoch {epoch}:\n\tTraining accuracy: {epoch_train_acc}\n\tValidation accuracy: {valid_acc}')

Epoch 0:
	Training accuracy: 0.621
	Validation accuracy: 0.701
Epoch 1:
	Training accuracy: 0.65
	Validation accuracy: 0.71
Epoch 2:
	Training accuracy: 0.503
	Validation accuracy: 0.5
Epoch 3:
	Training accuracy: 0.511
	Validation accuracy: 0.7
Epoch 4:
	Training accuracy: 0.524
	Validation accuracy: 0.5
Epoch 5:
	Training accuracy: 0.5
	Validation accuracy: 0.5
Epoch 6:
	Training accuracy: 0.5
	Validation accuracy: 0.5
